<a href="https://colab.research.google.com/github/maxenlee/Deep-Dive/blob/main/NLTKProcessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 5 NLTK

In [1]:
!pip install scikit-learn==1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import pandas as pd
import os
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from textblob import TextBlob, Word


from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import random
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')


## Importing File

In [ ]:
# Assuming 'url' is the path to your CSV file
csv_url = 'https://ddc-datascience.s3.amazonaws.com/Projects/Project.5-NLP/Data/NLP.csv'
df = pd.read_csv(csv_url)


In [ ]:
df.shape

In [ ]:
from urllib.parse import unquote
df['name'] = df['name'].apply(lambda x: unquote(x))

### Transforming the text in the text column


*   Tokenizing
*   removing punction with REGEX  
*   Removing stop words
*   lemmatizing
*   improved speed with memoization
*   caching lemmatization



In [ ]:
# import pandas as pd
# import re
# import string
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize
# import os
# import pickle

# # Ensure necessary NLTK data is downloaded
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

# # Initialize global components for text processing
# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()
# punctuation_removal_pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))

# # Load or initialize the lemmatization cache
# cache_file_path = 'lemmatization_cache.pkl'
# if os.path.exists(cache_file_path):
#     with open(cache_file_path, 'rb') as cache_file:
#         lemmatization_cache = pickle.load(cache_file)
# else:
#     lemmatization_cache = {}

# def preprocess_text_optimized(text):
#     global lemmatization_cache
#     text = punctuation_removal_pattern.sub('', text)
#     tokens = word_tokenize(text)
#     processed_tokens = []
#     for word in tokens:
#         word_lower = word.lower()
#         if word_lower not in stop_words:
#             if word_lower in lemmatization_cache:
#                 lemma = lemmatization_cache[word_lower]
#             else:
#                 lemma = lemmatizer.lemmatize(word_lower)
#                 lemmatization_cache[word_lower] = lemma
#             processed_tokens.append(lemma)
#     return " ".join(processed_tokens)


# class TextProcessor:
#     def __init__(self, optional_names_dict=None):
#         """
#         Initializes the TextProcessor with an optional dictionary for name removal.

#         Args:
#             optional_names_dict (dict): Optional. A dictionary mapping indexes to lists of names to remove from the text.
#         """
#         self.optional_names_dict = optional_names_dict or {}

#     def preprocess_dataframe(self, data, text_column_name='text'):
#         """
#         Preprocesses a DataFrame by removing specified names and applying text optimization techniques on a specified text column.

#         Args:
#             df (pd.DataFrame): The DataFrame containing the text to be processed.
#             text_column_name (str): The name of the column in the DataFrame that contains the text to process.

#         Returns:
#             pd.DataFrame: A DataFrame with the processed text in the specified column.
#         """
#         processed_texts = []
#         for index, row in data.iterrows():
#             text = row[text_column_name]
#             # Optional name removal
#             if index in self.optional_names_dict:
#                 for name in self.optional_names_dict[index]:
#                     text = re.sub(r'\b' + re.escape(name) + r'\b', '', text, flags=re.IGNORECASE)
#             # Apply optimized text processing
#             processed_text = preprocess_text_optimized(text)
#             processed_texts.append(processed_text)

#         # Example place to save the cache after processing is complete
#         with open(cache_file_path, 'wb') as cache_file:
#             pickle.dump(lemmatization_cache, cache_file)

#         # Update the DataFrame with processed texts
#         data[text_column_name] = processed_texts
#         return data

In [ ]:
# # Assuming 'df' is your DataFrame and 'name' is a column of names you want to optionally remove
# # Convert 'name' column to a dictionary mapping from index to a list of names (if there are multiple names to remove per index)
# optional_names_dict = df['name'].apply(lambda x: [x.lower()]).to_dict()

# # Initialize the TextProcessor with the optional names dictionary
# processor = TextProcessor(optional_names_dict=optional_names_dict)

# # Process the DataFrame, specifying the text column to process
# processed_df = processor.preprocess_dataframe(df, text_column_name='text')
# # Assuming 'df' is your original DataFrame and 'text_column_name' is the name of the column containing text


In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

# Ensure that NLTK resources are downloaded (do this once)
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# Initialize the lemmatizer and stop words list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Cache for memoization
lemmatization_cache = {}

def lemmatize_with_cache(word):
    # If the word has already been lemmatized, return the result from the cache
    if word in lemmatization_cache:
        return lemmatization_cache[word]
    # Otherwise, lemmatize the word and store the result in the cache
    else:
        lemmatized_word = lemmatizer.lemmatize(word)
        lemmatization_cache[word] = lemmatized_word
        return lemmatized_word

def get_proper_nouns(text):
    """Extract proper nouns from a given text."""
    proper_nouns = []
    tagged_tokens = pos_tag(word_tokenize(text))
    for word, tag in tagged_tokens:
        if tag == 'NNP' or tag == 'NNPS':  # Proper nouns
            proper_nouns.append(word.lower())
    return set(proper_nouns)

def process_text(df,on,optional_dict = {}):
    processed_texts = []

    for index, row in df.iterrows():
        # Extract proper nouns from the 'name' column to exclude from the 'text'
        proper_nouns = get_proper_nouns(row[optional_dict])

        # Tokenize and remove punctuation
        tokens = word_tokenize(row[on])
        tokens = [re.sub(r'\W+', '', token) for token in tokens if re.sub(r'\W+', '', token)]

        # Remove stop words, proper nouns, and lemmatize
        tokens = [lemmatize_with_cache(token.lower()) for token in tokens if token.lower() not in stop_words and token.lower() not in proper_nouns]

        # Rejoin tokens into a single string
        processed_text = ' '.join(tokens)
        processed_texts.append(processed_text)

    # Return a new DataFrame with processed texts
    return pd.DataFrame({'text': processed_texts})


In [ ]:
# Assuming 'df' is your DataFrame with a 'text' column
processed_df = process_text(df,on = 'text', optional_dict= 'name')


# TF-IDF


In [ ]:
processed_df.head()

In [ ]:
df['text'] = processed_df['text']

In [ ]:
df['text']

In [ ]:
og_size = sum(df['text'].apply(lambda x: len(x)))
og_size

In [ ]:
processed_size = sum(processed_df['text'].apply(lambda x: len(x)))
processed_size

In [ ]:
og_size - processed_size

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tf_idf_matrix = tfidf_vectorizer.fit_transform(processed_df['text'])

### Fitting K Nearest neighbors

In [ ]:
nearest_neighbors = NearestNeighbors().fit(tf_idf_matrix)

In [ ]:
np.random.seed(24)
page = random.randrange(0,len(df))
wiki = tf_idf_matrix[page]


In [ ]:
page

In [ ]:
wiki

## nearest neighbors in DB Pedia

In [ ]:
distances, indices = nearest_neighbors.kneighbors(X = wiki , n_neighbors=11 )

# Retrieve the original texts for the nearest neighbors
# nearest_texts = original_texts.iloc[indices.flatten()]



In [ ]:
neighbors_index = list(indices[0][:])

In [ ]:
neighbors_distance = list(distances[0][:])

In [ ]:
zipped = list(zip(neighbors_index,neighbors_distance))

In [ ]:
zipped

In [ ]:
db_name = df.iloc[page]['name']

In [ ]:
df_DBneighbors = df.iloc[neighbors_index]


In [ ]:
DB_nn = pd.Series(distances[0][:],name = 'distances' , index= indices[0][:])
# df_Wiki_Neighbors = pd.concat([df_Wiki_Neighbors,wiki_nn], axis = 0)
df_DBneighbors = df_DBneighbors.join(DB_nn,)

In [ ]:
df_DBneighbors

### Sentiment scores for each neighbor

In [ ]:
df_DBneighbors.loc[:, 'sentiment'] = df_DBneighbors['text'].apply(lambda x: TextBlob(x).sentiment)

In [ ]:
df_dict={}
df_dict['DB'] = df_DBneighbors

In [ ]:
df_dict['DB']

# Part 2: The SQL
Of the 10 Nearest Neighbors, Find their Whole Wiki Page and Order Their Indecies by the Whole Page.

In [ ]:
%%capture
!pip3 install wikipedia-api

In [ ]:
import wikipediaapi
wikipedia = wikipediaapi.Wikipedia(user_agent = 'abooboo')

In [ ]:
wiki

In [ ]:
Person = db_name
person_text = wikipedia.page(Person)
person_text.title
person_text.text

In [ ]:
df_Wiki_Neighbors = pd.DataFrame()
df_Wiki_Neighbors['name'] = df_DBneighbors['name'].apply(lambda x: wikipedia.page(x).title)
df_Wiki_Neighbors['text'] = df_Wiki_Neighbors['name'].apply(lambda x: wikipedia.page(x).text )
df_Wiki_Neighbors['sentiment'] = df_Wiki_Neighbors['text'].apply(lambda x: TextBlob(x).sentiment)

In [ ]:
df_Wiki_Neighbors = df_Wiki_Neighbors.reset_index(drop=True)

In [ ]:

processed_wiki_df = process_text(df_Wiki_Neighbors,on = 'text', optional_dict= 'name')


In [ ]:
processed_wiki_df = processed_wiki_df.reset_index(drop = True)

In [ ]:
processed_wiki_df

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tf_idf_matrix = tfidf_vectorizer.fit_transform(processed_wiki_df['text'])
nearest_neighbors = NearestNeighbors().fit(tf_idf_matrix)

In [ ]:
page = 0
wiki = tf_idf_matrix[page]
distances, indices = nearest_neighbors.kneighbors(X = wiki , n_neighbors=11 )

In [ ]:
distances[0]

In [ ]:
wiki_nn = pd.Series(distances[0][:],name = 'distances' , index= indices[0][:])
# df_Wiki_Neighbors = pd.concat([df_Wiki_Neighbors,wiki_nn], axis = 0)
df_Wiki_Neighbors = df_Wiki_Neighbors.join(wiki_nn,)

In [ ]:
df_Wiki_Neighbors

# Part3 Interactivity


In [ ]:

# # Assuming 'df' is your DataFrame and 'name' is a column of names you want to optionally remove
# # Convert 'name' column to a dictionary mapping from index to a list of names (if there are multiple names to remove per index)
# optional_names_dict = df['name'].apply(lambda x: [x.lower()]).to_dict()

# # Initialize the TextProcessor with the optional names dictionary
# processor = TextProcessor(optional_names_dict=optional_names_dict)

# # Process the DataFrame, specifying the text column to process
# processed_df = processor.preprocess_dataframe(df, text_column_name='text')

# # 'processed_df' now contains the optimized, processed text

In [ ]:
# Necessary imports
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from textblob import TextBlob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

# Assuming TextProcessor is a custom class you've defined elsewhere
# from your_custom_module import TextProcessor

def get_nearest_neighbors(df, page, k=10):
    """
    Returns the nearest neighbors from DB Pedia and Wiki for a given page.

    Args:
      df: The DataFrame containing the processed text.
      page: The index of the page to find nearest neighbors for.
      k: The number of nearest neighbors to return.

    Returns:
      A dictionary containing DataFrames of nearest neighbors from DB Pedia and Wiki.
    """

    # Ensure the necessary libraries are imported above


    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tf_idf_matrix = tfidf_vectorizer.fit_transform(df['text'])
    nearest_neighbors = NearestNeighbors().fit(tf_idf_matrix)
    wiki = tf_idf_matrix[page]
    db_distances, db_indices = nearest_neighbors.kneighbors(X=wiki, n_neighbors=k+1)

    # DB DataFrame
    df_DBneighbors = df.iloc[db_indices[0]]
    df_DBneighbors = df_DBneighbors.drop(columns=['URI'], errors='ignore')
    df_DBneighbors['sentiment'] = df_DBneighbors['text'].apply(lambda x: TextBlob(x).sentiment)
    df_DBneighbors['distances'] = db_distances[0]


    # Wiki DataFrame
    df_DBneighbors['wiki_name'] = df_DBneighbors['name'].apply(lambda x: wikipedia.page(x).title)
    df_DBneighbors['wiki_text'] = df_DBneighbors['wiki_name'].apply(lambda x: wikipedia.page(x).text)  # Use summary for brevity
    df_DBneighbors['wiki_sentiment'] = df_DBneighbors['wiki_text'].apply(lambda x: TextBlob(x).sentiment)
    wiki_text = process_text(df_DBneighbors,on = 'wiki_text', optional_dict= 'wiki_name')
    # Ensure the necessary libraries are imported above

    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tf_idf_matrix = tfidf_vectorizer.fit_transform(wiki_text['text'])
    nearest_neighbors = NearestNeighbors().fit(tf_idf_matrix)
    wiki = tf_idf_matrix[0]
    db_distances, db_indices = nearest_neighbors.kneighbors(X=wiki, n_neighbors=k+1)
    df_DBneighbors['wiki_distances'] = db_distances[0]

    # df_DBneighbors

    df_DBneighbors['pol1'] = df_DBneighbors['sentiment'].apply(lambda x: x[0])
    df_DBneighbors['sub1'] = df_DBneighbors['sentiment'].apply(lambda x: x[1])

    df_DBneighbors['pol2'] = df_DBneighbors['wiki_sentiment'].apply(lambda x: x[0])
    df_DBneighbors['sub2'] = df_DBneighbors['wiki_sentiment'].apply(lambda x: x[1])

    output = df_DBneighbors


    fig, ax = plt.subplots(figsize=(12, 6))  # Increase figure width

    # Define a list of markers to cycle through
    markers = ['o', 'v', '^', '<', '>', 's', 'p', '*', '+', 'x','D']

    # Placeholder for custom legend icons
    legend_icons = []

    mark_i = 0
    # Loop through the DataFrame and plot each point with a different marker
    for i, row in output.iterrows():
        scatter1 = ax.scatter(row['pol1'], row['sub1'], color='blue', label='DB' if i == 0 else "", marker=markers[mark_i])
        scatter2 = ax.scatter(row['pol2'], row['sub2'], color='red', label='wiki' if i == 0 else "", marker=markers[mark_i])
        # Create a custom legend icon for each name
        legend_icon = mlines.Line2D([], [], color='black', marker=markers[mark_i], linestyle='None', markersize=10, label=row['name'])
        legend_icons.append(legend_icon)
        mark_i += 1

    # Set labels and title
    ax.set_xlabel('Polarity')
    ax.set_ylabel('Subjectivity')
    ax.set_title('Sentiment Analysis: Polarity vs. Subjectivity')

    # Create the first legend manually and specify its position outside the plot area
    first_legend = ax.legend(handles=legend_icons, loc='upper left', bbox_to_anchor=(1, 1), title='Names')
    # Add the first legend manually


    second_legend = ax.legend([scatter1, scatter2], ['DB', 'wiki'], loc='lower left', bbox_to_anchor=(1, 0))

    # Create and add the second legend for the scatter plot colors/labels, also outside
    plt.legend(handles=legend_icons, loc='upper left', bbox_to_anchor=(1, 1), title='Names')
    ax.add_artist(second_legend)
    plt.subplots_adjust(right=.75)  # Adjust subplot to create more space on the right

    plt.show()

    return df_DBneighbors


# Example usage (assuming 'df' and 'condition' are defined in your context)
search = 'Franz Rottensteiner'
condition = df[df['name'] == search].index[0]
output = get_nearest_neighbors(df, page=condition)


In [ ]:
output

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.lines as mlines

# fig, ax = plt.subplots(figsize=(12, 6))  # Increase figure width

# # Define a list of markers to cycle through
# markers = ['o', 'v', '^', '<', '>', 's', 'p', '*', '+', 'x']

# # Placeholder for custom legend icons
# legend_icons = []

# # Loop through the DataFrame and plot each point with a different marker
# for i, row in output.iterrows():
#     scatter1 = ax.scatter(row['pol1'], row['sub1'], color='blue', label='DB' if i == 0 else "", marker=markers[i % len(markers)])
#     scatter2 = ax.scatter(row['pol2'], row['sub2'], color='red', label='wiki' if i == 0 else "", marker=markers[i % len(markers)])
#     # Create a custom legend icon for each name
#     legend_icon = mlines.Line2D([], [], color='black', marker=markers[i % len(markers)], linestyle='None', markersize=10, label=row['name'])
#     legend_icons.append(legend_icon)

# # Set labels and title
# ax.set_xlabel('Polarity')
# ax.set_ylabel('Subjectivity')
# ax.set_title('Sentiment Analysis: Polarity vs. Subjectivity')

# # Create the first legend manually and specify its position outside the plot area
# first_legend = ax.legend(handles=legend_icons, loc='upper left', bbox_to_anchor=(1, 1), title='Names')
# # Add the first legend manually


# second_legend = ax.legend([scatter1, scatter2], ['DB', 'wiki'], loc='lower left', bbox_to_anchor=(1, 0))

# # Create and add the second legend for the scatter plot colors/labels, also outside
# plt.legend(handles=legend_icons, loc='upper left', bbox_to_anchor=(1, 1), title='Names')
# ax.add_artist(second_legend)
# plt.subplots_adjust(right=.75)  # Adjust subplot to create more space on the right

# plt.show()


# USER INPUT

Exact match

In [ ]:
def name_search():
  search = input("Enter a name from the DataFrame: ")
  print('loading...')
  condition = df[df['name'] == search].index[0]
  output = get_nearest_neighbors(df, page=condition)
  print('DING!')

In [ ]:
my_neighbors = name_search()

In [ ]:
import numpy as np
import pandas as pd
import textdistance


def get_closest_names(df, name, k=5):
    """
    Returns the k closest names to the given name in the DataFrame using Levenshtein distance.

    Args:
        df: The DataFrame containing the names.
        name: The name to find the closest names to.
        k: The number of closest names to return.

    Returns:
        A list of the k closest names.
    """

    # Calculate the Levenshtein distance between the given name and all other names
    distances = df['name'].apply(lambda x: textdistance.levenshtein(name.lower(), x.lower()))

    # Sort the distances in ascending order and get the indices of the k closest names
    sorted_indices = np.argsort(distances)[:k]

    # Return the names corresponding to the closest indices
    return df['name'].iloc[sorted_indices].tolist()



In [ ]:
# Example usagew
search = input("Enter a name from the DataFrame: ")
closest_names = get_closest_names(df, search)
print("The 5 closest names to", search, "are:", closest_names)


In [ ]:
import pandas as pd
import numpy as np
import textdistance

# Sample DataFrame
# df = pd.DataFrame({'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve']})

def get_closest_names(df, name, k=5):
    """
    Returns the k closest names to the given name in the DataFrame using Levenshtein distance.

    Args:
        df: The DataFrame containing the names.
        name: The name to find the closest names to.
        k: The number of closest names to return.

    Returns:
        A list of the k closest names.
    """
    distances = df['name'].apply(lambda x: textdistance.levenshtein(name.lower(), x.lower()))
    sorted_indices = np.argsort(distances)[:k]
    return df['name'].iloc[sorted_indices].tolist()

def execute_function_with_name(search):
    """
    Placeholder for executing a function with the selected name.

    Args:
        name: The selected name to use in the function.
    """
    print('loading...')
    condition = df[df['name'] == search].index[0]
    output = get_nearest_neighbors(df, page=condition)
    print('DING!')
    # Placeholder functionality
    print(f"Function executed with {search}.")


In [ ]:
# Interactive process
search = input("Enter a name from the DataFrame: ")
closest_names = get_closest_names(df, search)
print("The closest names to", search, "are:")
for index, name in enumerate(closest_names, start=1):
    print(f"{index}. {name}")

selected_index = int(input("Select a name by entering the corresponding number: ")) - 1
if 0 <= selected_index < len(closest_names):
    selected_name = closest_names[selected_index]
    print(f"You selected: {selected_name}")
    execute_function_with_name(selected_name)
else:
    print("Invalid selection. Please run the process again and select a valid number.")


In [ ]:
m= ['o', 'v', '^', '<', '>', 's', 'p', '*', '+', 'x','^']

In [ ]:
counter = 0
for i,k in output.iterrows():
  print(m[counter])
  counter +=1

In [ ]:
m[0]